CNN의 가장 마지막 단계인 SoftMax, LinearClassifier, Perceptron 부분을 먼저 구현한다. <br>
참고로 이것은 성능이 좋지 못한데, 그래서 그 앞부분에 전 결합층, 드롭아웃 계층, 합성곱 .. 등이 추가된다. <br>
이것은 다음에 업데이트 한다. 

**머신러닝 모델의 3단계**<br>
1. 수식을 생각한다 - 주어진 데이터를 기반으로 미지의 데이터를 예측하는 수식을 생각한다. 
2. 오차함수를 준비한다. - 수식에 포함된 파라미터의 좋고 나쁨을 판단하는 오차 함수를 준비한다. 
3. 파라미터 값을 결정한다. - 오차 함수를 최소화할 수 있도록 파라미터의 값을 결정한다. 

텐서플로를 이용한 maximum likelihood estimation <br>

In [12]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import numpy as np 
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame, Series

트레이닝 세트 데이터를 준비한다. 

In [2]:
np.random.seed(20160512)

n0, mu0, variance0 = 20, [10, 11], 20
data0 = multivariate_normal(mu0, np.eye(2)*variance0 ,n0) # multivariate_normal(중심좌표, variances, 데이터 갯수)
df0 = DataFrame(data0, columns=['x1','x2'])
df0['t'] = 0

n1, mu1, variance1 = 15, [18, 20], 22
data1 = multivariate_normal(mu1, np.eye(2)*variance1 ,n1)
df1 = DataFrame(data1, columns=['x1','x2'])
df1['t'] = 1

df = pd.concat([df0, df1], ignore_index=True) #pd.concat()
train_set = df.reindex(permutation(df.index)).reset_index(drop=True) #permutaion(array): array를 복사해서 셔플한다. np.random.shuffle([1,2,3,4,5]) ==> [3,2,4,5,1]
                                                                     # dataframe.reindex(array): dataframe을 array를 이용하여 복사한다. 
    
print(df0.shape)
print(df1.shape)
print(train_set.shape)
train_set[:10]

(20, 3)
(15, 3)
(35, 3)


,x1,x2,t
0,20.729880,18.209359,1
1,16.503919,14.685085,0
2,5.508661,17.426775,0
3,9.167047,9.178837,0
4,8.640423,9.561952,0
5,10.500988,17.002584,0
6,16.484766,22.232997,1
7,6.979059,8.180596,0
8,11.701339,0.996734,0
9,21.367990,18.712309,1


In [3]:
#훈련에 사용할 데이터셋object has 준비
data = train_set[['x1','x2']].values
print(data.shape)
data[:5]

(35, 2)


array([[20.72988036, 18.20935928],
       [16.50391894, 14.68508536],
       [ 5.50866102, 17.42677464],
       [ 9.16704708,  9.17883747],
       [ 8.6404225 ,  9.56195153]])

In [4]:
data_t = train_set[['t']].values
print(data_t.shape)
data_t[:10]

(35, 1)


array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1]])

In [5]:
#그래프를 준비한다. 
X = tf.placeholder(tf.float32, shape=(None,2))
Y = tf.placeholder(tf.float32, shape=(None,1))
M = tf.Variable([[0.0], [0.0]])
M0 = tf.Variable([[0.0]])
Z = tf.matmul(X,M)+M0
p = tf.sigmoid(Z)

In [6]:
#오차함수를 준비한다. 
loss = - tf.reduce_sum(Y*tf.log(p)+(1-Y)*tf.log(1-p))

In [7]:
#트레이닝 알고리즘을 정한다. 
train_one_step = tf.train.AdamOptimizer().minimize(loss)

In [8]:
#중간중간 결과를 출력할 값을 정한다. 
# loss, accuracy 
accuracy = tf.reduce_mean(
                tf.cast( 
                    tf.equal(
                        tf.sign(p-0.5), 
                        tf.sign(Y-0.5))
                    , tf.float32)
            )

In [9]:
#Session 생성, 트레이닝object has
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(20000):
    sess.run(train_one_step, feed_dict={X:data, Y:data_t})
    if i%2000 ==0:
        [loss_value, acc_value]= sess.run([loss, accuracy], feed_dict={X:data, Y:data_t})
        #print("i={}".format(i))
        print("loss:{}  acc_value:{}".format(loss_value, acc_value))

loss:24.15797233581543  acc_value:0.4285714328289032
loss:15.162978172302246  acc_value:0.8857142925262451
loss:10.770998001098633  acc_value:0.9142857193946838
loss:8.196752548217773  acc_value:0.9714285731315613
loss:6.575472831726074  acc_value:0.9714285731315613
loss:5.511539936065674  acc_value:0.9428571462631226
loss:4.797740936279297  acc_value:0.9428571462631226
loss:4.31400203704834  acc_value:0.9428571462631226
loss:3.9861483573913574  acc_value:0.9428571462631226
loss:3.766432285308838  acc_value:0.9428571462631226


In [10]:
# # 그래프로 결과를 그린다. 
# train_set_0 = train_set[train_set.t==0]
# train_set_1 = train_set[train_set.t==1]
# fig = plt.figure(figsize=(6,6))
# subplot = fig.add_subplot(1,1,1)
# subplot.set_xlim([0,30])
# subplot.set_ylim([0,30])
# subplot.scatter(train_set_1.x1, train_set_1.x2, marker='x')
# subplot.scatter(train_set_0.x1, train_set_0.x2, marker='o')

# linex = np.linspace(0,30,10)

In [11]:
#테스트 세트를 이용한 검증 
